The Notebook presents a runthrough the Facial Keypoint Detection Challenge hosted on Kaggle

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

from matplotlib import pyplot as plt
%matplotlib inline 

In [ ]:
base_dir='/kaggle/input/facial-keypoints-detection/'
train_dir_zip=base_dir+'training.zip'
test_dir_zip=base_dir+'test.zip'

In [ ]:
from zipfile import ZipFile
with ZipFile(train_dir_zip,'r') as zipObj:
    zipObj.extractall('.')
    print("Train Archive unzipped")
with ZipFile(test_dir_zip,'r') as zipObj:
    zipObj.extractall('.')
    print("Test Archive unzipped")
    
train_dir='./training.csv'
test_dir='./test.csv'
train=pd.read_csv(train_dir)
test=pd.read_csv(test_dir)

#print(train)
train.head()

In [ ]:
print("Size of dataframe: "+str(len(train))+'x'+str(len(train.columns))+'\n')
#print(train.info())

train_to_aug=train.dropna()
train_to_aug=train_to_aug.reset_index(drop=True) #Pro augmenatace

train = train.fillna(method='ffill')
print("Size of dataframe after: "+str(len(train))+'x'+str(len(train.columns))+'\n')

#print("After droppping all the rows with any NA in column\n")
#print("Size = "+str(len(train))+'x'+str(len(train.columns)))

In [ ]:
X=[]
Y=[]
for img in train['Image']:
    dato = np.asarray(img.split(),dtype=float).reshape(96,96,1)
    #dato = dato/255
    X.append(dato)
    #print(X[0][0])
    #break
    
X = np.reshape(X,(-1,96,96,1))
X = np.asarray(X).astype('float32')
    
for i in range(len((train))):
    Y.append(np.asarray(train.iloc[i][0:30].to_numpy()))
Y = np.asarray(Y).astype('float32')

print(X.shape)
print(Y.shape)

X_aug = []
Y_aug = []

for img in train_to_aug['Image']:
    dato = np.asarray(img.split(),dtype=float).reshape(96,96,1)
    X_aug.append(dato)
    
X_aug = np.reshape(X_aug,(-1,96,96,1))
X_aug = np.asarray(X_aug).astype('float32')
    
for i in range(len((train_to_aug))):
    Y_aug.append(np.asarray(train_to_aug.iloc[i][0:30].to_numpy()))
Y_aug = np.asarray(Y_aug).astype('float32')

In [ ]:
disp=8

fig,axes=plt.subplots((disp+3)//4,4,figsize=(15,10))
                    
for i in range(disp):
    axes[i//4,i%4].imshow(X[i].reshape(96,96),cmap='gray')
    axes[i//4,i%4].scatter([train[train.columns[2*j]][i] for j in range(15)],[train[train.columns[2*j+1]][i] for j in range(15)],s=10,c='r')

In [ ]:
import cv2
from math import sin, cos, pi

def aug_rotation(X, Y):
    rotation_angles = [10,20,30]
    
    rotated_images = []
    rotated_keypoints = []
    
    for angle in rotation_angles:
        for angle in [angle, -angle]:
            rot = cv2.getRotationMatrix2D((48,48), angle, 1.)
            angle_rad = -angle*pi/180.
            
            for image in X:
                rotated_image = cv2.warpAffine(image, rot, (96,96), flags=cv2.INTER_CUBIC)
                rotated_images.append(rotated_image)
                
            for keypoint in Y:
                rotated_keypoint = keypoint - 48.
                
                for idx in range(0, len(rotated_keypoint), 2):
                    rotated_keypoint[idx] = rotated_keypoint[idx]*cos(angle_rad)-rotated_keypoint[idx+1]*sin(angle_rad)
                    rotated_keypoint[idx+1] = rotated_keypoint[idx]*sin(angle_rad)+rotated_keypoint[idx+1]*cos(angle_rad)
                rotated_keypoint += 48.   
                rotated_keypoints.append(rotated_keypoint)
            
    return np.reshape(rotated_images,(-1,96,96,1)), rotated_keypoints


#X_rot , Y_rot = aug_rotation(X[0],Y[0])
#fig, axis = plt.subplots()
#img_rot = X_rot.reshape(96,96)
#axis.imshow(img_rot='gray')
#axis.scatter(keypoint[0::2], keypoint[1::2], s=10, c='r')
#plot_sample(X_rot, Y_rot, axis, "Rotation Augmentation")


X_train_rot, Y_train_rot = aug_rotation(X_aug,Y_aug)
fig, axis = plt.subplots()
img_rot = X_train_rot[1].reshape(96,96)
axis.imshow(img_rot, cmap='gray')
axis.scatter(Y_train_rot[1][0::2], Y_train_rot[1][1::2], s=10, c='r')



X = np.concatenate((X, X_train_rot))
Y = np.concatenate((Y, Y_train_rot))

print(X.shape)
print(Y.shape)

In [ ]:
def aug_shift(X, Y):
    pixel_shifts = [5,15]
    shifted_images = []
    shifted_keypoints = []
    for shift in pixel_shifts:    
        for (shift_x,shift_y) in [(-shift,-shift),(-shift,shift),(shift,-shift),(shift,shift)]:
            sh = np.float32([[1,0,shift_x],[0,1,shift_y]])
            
            for image, keypoint in zip(X, Y):
                shifted_image = cv2.warpAffine(image, sh, (96,96), flags=cv2.INTER_CUBIC)
                shifted_keypoint = np.array([(point+shift_x) if idx%2==0 else (point+shift_y) for idx, point in enumerate(keypoint)])
                
                if np.all(0.0<shifted_keypoint) and np.all(shifted_keypoint<96.0):
                    shifted_images.append(shifted_image.reshape(96,96,1))
                    shifted_keypoints.append(shifted_keypoint)
    shifted_keypoints = np.clip(shifted_keypoints,0.0,96.0)
    return shifted_images, shifted_keypoints

X_train_shift, Y_train_shift = aug_shift(X_aug,Y_aug)
fig, axis = plt.subplots()
img_sh = X_train_shift[1].reshape(96,96)
axis.imshow(img_sh, cmap='gray')
axis.scatter(Y_train_shift[1][0::2], Y_train_shift[1][1::2], s=10, c='r')



X = np.concatenate((X, X_train_shift))
Y = np.concatenate((Y, Y_train_shift))

print(X.shape)
print(Y.shape)

In [ ]:
def aug_brightness(X, Y):
    altered_brightness_images = []
    inc_brightness_images = np.clip(X*1.2, 0, 255)    
    dec_brightness_images = np.clip(X*0.6, 0, 255)    
    altered_brightness_images.extend(inc_brightness_images)
    altered_brightness_images.extend(dec_brightness_images)
    return altered_brightness_images, np.concatenate((Y, Y))

X_train_brightness, Y_train_brightness = aug_brightness(X_aug,Y_aug)
fig, axis = plt.subplots()
img_br = X_train_brightness[1].reshape(96,96)
axis.imshow(img_br, cmap='gray')
axis.scatter(Y_train_brightness[1][0::2], Y_train_brightness[1][1::2], s=10, c='r')

#print(X_train_brightness[2])

X = np.concatenate((X, X_train_brightness))
Y = np.concatenate((Y, Y_train_brightness))

print(X.shape)
print(Y.shape)


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1)

#print(X_train[0])
#print(Y_train[0])
#print('')

print(len(X_train))
print(len(Y_train))
print('')
print(len(X_test))
print(len(Y_test))

In [ ]:
import tensorflow
#from tensorflow.keras.models import Sequential, Model
#from tensorflow.keras.layers import BatchNormalization, Flatten, Dense, Dropout, Conv2D, MaxPool2D, LeakyReLU

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Conv2D, LeakyReLU, GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.models import Sequential

model = Sequential()
pretrained_model = ResNet50(input_shape=(96,96,3), include_top=False, weights='imagenet')
pretrained_model.trainable = True

model.add(Conv2D(3, (1,1), padding='same', input_shape=(96,96,1)))
model.add(LeakyReLU(alpha=0.1))
model.add(pretrained_model)
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.1))
model.add(Dense(30))
model.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
earlyStopping = EarlyStopping(monitor='loss', patience=30, mode='min',
                             baseline=None)

rlp = ReduceLROnPlateau(monitor='val_loss', factor=0.7, patience=5, min_lr=1e-15, mode='min', verbose=1)

model.compile(optimizer='Adam', loss='mse', metrics=['accuracy'])
history=model.fit(X_train, Y_train, epochs=200 ,batch_size=32,validation_data=(X_test,Y_test),callbacks=[earlyStopping, rlp])

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Loss vs Epoch')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.ylim(0,15)
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
Test_Data=[]
for img in test['Image']:
    Test_Data.append(np.asarray(img.split(),dtype=float).reshape(96,96,1))
Test_Data=np.reshape(Test_Data,(-1,96,96,1))
Test_Data = np.asarray(Test_Data).astype('float32')

In [ ]:
Pred=model.predict(Test_Data)

In [ ]:
disp=8

fig,axes=plt.subplots((disp+3)//4,4,figsize=(15,10))
                    
for i in range(disp):
    axes[i//4,i%4].imshow(Test_Data[i].reshape(96,96),cmap='gray')
    axes[i//4,i%4].scatter([Pred[i][2*j] for j in range(15)],[Pred[i][2*j+1] for j in range(15)],s=10,c='r')

In [ ]:
idtable=pd.read_csv(base_dir+'IdLookupTable.csv')
rowId=list(idtable['RowId'])
imageId=list(idtable['ImageId'])
featureHead=list(train.columns.values)
featureIndex=[featureHead.index(feature) for feature in idtable['FeatureName']]

loc=[]
for index,imgId in zip(featureIndex,imageId):
    loc.append(Pred[imgId-1][index])
subm=pd.DataFrame({'RowId':rowId,'Location':loc})
subm.head()

In [ ]:
subm.to_csv('submission.csv',index = False)